In [1]:
number_of_shots = 5

In [2]:
experiment_name = "2024.05.07-05_Mixtral_7B_instruct_API_5-few-shots_SEMANTIC_SIMILARITY"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 0---
1 6099 0  ---SDG 0--- 

Explanation:
The text is about a model's photo shoot for the Sports Illustrated Swimsuit issue in the Maldives. It does not contain any information related to the Sustainable Development Goals. Therefore, the appropriate classification is "---SDG 0---".
2 6471 0  ---SDG 16---
3 1786 0  ---SDG 0---
4 2143 0  ---SDG 9---
5 3377 0  ---SDG 0---

The text provided is about Kelly Clarkson's new album, her personal life, and her reactions to some claims made about her talk show. While the topic touches on entertainment and media, it does not directly relate to any specific Sustainable Development Goal.
6 5589 0  ---SDG 0---

The text does not fall under any specific Sustainable Development Goal. It discusses the development and release of a horror game, Alan Wake 2, and related business decisions. It does not address issues related to poverty, hunger, health, education, gender equality, clean water and sanitation, affordable and clean energy, dece

56 2463 0  ---SDG 0---
57 193 0  ---SDG 0---
58 3497 0  ---SDG 3---
59 3751 0  ---SDG 3---

(Note: SDG 3 is for "Good Health and Well-being")
60 6573 1  ---SDG 1---
61 357 1  ---SDG 1---
62 175 1  ---SDG 1---

The text discusses the issue of poverty, specifically extreme poverty, and the potential for reducing it by 2030. This aligns with the focus of SDG 1, which is to end poverty in all its forms everywhere. The text highlights the current state of poverty in certain regions, such as sub-Saharan Africa, and the potential impact of economic growth on poverty reduction. This focus on poverty reduction and the specific mention of SDG 1's goal of ending poverty make it clear that this text falls under ---SDG 1---.
63 1446 1  ---SDG 1--- \\
This text discusses the relationship between employment and poverty, particularly for children and households. It highlights the varying risks of poverty associated with different employment situations across different countries, suggesting a focus on 

98 3745 1  ---SDG 1---

The text discusses the factors contributing to the increase of child poverty, specifically changes in consumer prices and reduction in income from self-employment in the non-agriculture sector. It highlights non-food items as the main channel through which children are affected by the crisis. The text also mentions the impact of falling wages on child poverty, with a more significant effect in Cameroon. The changes in the informal wage sector affecting children more than the formal sector is also noted. All these factors relate to Goal 1: No Poverty, as they directly discuss the issue of poverty and potential solutions to alleviate it.
99 1104 1  ---SDG 1---
100 2577 1  ---SDG 1---
101 5269 1  ---SDG 1---
102 3166 1  ---SDG 1---
103 4534 1  ---SDG 1---
104 1733 1  ---SDG 8---
105 2694 1  ---SDG 1---
 (No Poverty)

The text discusses issues related to child poverty, income distribution, and the role of policies and societal factors in addressing these issues. The

144 2727 2  ---SDG 2--- \\
This text discusses the need for a more radical and integrated policy approach to promote sustainable food production and reduce poverty, hunger, and environmental destruction. It highlights the importance of scaling up innovations, disseminating knowledge, building capacity among farmers, fostering trust, and reducing risks associated with new technology and agricultural practices. These themes align closely with SDG 2, which aims to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture.
145 869 2  ---SDG 2---
146 3205 2  ---SDG 2---

The text discusses issues related to food security, including the demand for land to grow staple foods and the vulnerability of certain regions to food price volatility. SDG 2 aims to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture.
147 6506 2  ---SDG 2--- (Sustainable Development Goal 2: Zero Hunger) as the text is about agricultural policies

188 5625 3  ---SDG 3---
189 3913 3  ---SDG 3---
190 1122 3  ---SDG 3---
191 6305 3  ---SDG 3--- (Goal 3 is "Good Health and Well-being")
192 1655 3  ---SDG 3---
193 1028 3  ---SDG 3---
194 6378 3  ---SDG 3---
195 1869 3  ---SDG 3---
196 1372 3  ---SDG 3---
197 3450 3  ---SDG 3---
198 1807 3  ---SDG 3---
199 1124 3  ---SDG 3---

The text emphasizes the importance of reducing inequalities in health policies, particularly for the most economically vulnerable children and their families. This aligns with SDG 3, which aims to ensure healthy lives and promote well-being for all at all ages.
200 1131 3  ---SDG 3---
201 4254 3  ---SDG 3---
202 5220 3  ---SDG 3---
203 4763 3  ---SDG 3---
204 4400 3  ---SDG 3---
205 4023 3  ---SDG 3---

The text discusses the progress made in preventing measles outbreaks through the use of pneumococcal conjugate vaccine (PCV), but notes that coverage is still insufficient and many countries, particularly middle-income countries, are lagging behind in accessing t

...

738 5570 12  ---SDG 12---

(Reasoning: The text discusses the implementation of national legislation regarding waste management and recycling, with a focus on the EU and related countries. This aligns closely with SDG 12, which aims to ensure sustainable consumption and production patterns, including through responsible management of chemicals and waste.)
739 4639 12  ---SDG 12---
740 5426 12  ---SDG 11---

The text discusses the environmental impact of motor vehicles throughout their life cycle, including resource extraction, production, use, and end-of-life disposal. SDG 11 focuses on making cities and human settlements inclusive, safe, resilient, and sustainable, which includes reducing the environmental impact of transportation.
741 6128 12  ---SDG 12---

(The text is about corporate social responsibility (CSR) and sustainable development, which aligns with SDG 12: Responsible Consumption and Production.)
742 2486 12  ---SDG 12---

Explanation:

The text emphasizes the importance o

793 2575 13  ---SDG 13---
794 5881 13  ---SDG 13---
795 2540 13  ---SDG 13---
796 6671 13  ---SDG 13---
797 4343 13  ---SDG 13---
798 1964 13  ---SDG 5--- (This refers to Gender Equality, one of the SDGs, as the text highlights the importance of women's involvement in community-based forest governance.)
799 1620 13  ---SDG 13---
800 3379 13  ---SDG 13---
(The text discusses the role of the UNFCCC in the climate regime and its interaction with other institutions for climate action, which aligns with SDG 13: Climate Action.)
801 4843 13  ---SDG 13---
802 6675 13  ---SDG 13---

The text discusses the effectiveness of climate finance interventions and the importance of mobilizing external resources for such interventions. While it touches upon various aspects of development and resource mobilization, the primary focus is on climate finance, which aligns with SDG 13: Climate Action.
803 6696 13  ---SDG 13---
804 5030 13  ---SDG 13---
805 5047 13  ---SDG 13--- \\
The text discusses the impor

857 4311 14  ---SDG 14---
858 3448 14  ---SDG 14---
859 3360 14  ---SDG 14---
860 5600 14  ---SDG 14---
861 6434 14  ---SDG 14---
862 5450 14  ---SDG 2---

The text is about the economic pressure in the industry leading to adaptation and evolution of operations to maximize economic return. It also mentions about unrestricted access to the market for imports of species not domestically produced, and stable per capita consumption with growth largely dependent on population trends. While the text does touch upon the food industry, it does not specifically mention any goals related to sustainable fisheries or oceans. SDG 2, which aims to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture, is the most fitting goal for this text.
863 6225 14  ---SDG 14---
864 4600 14  ---SDG 14---
865 2639 14  ---SDG 14---
866 2234 14  ---SDG 14---
867 2277 14  ---SDG 14---
868 42 14  ---SDG 14---

Explanation:

The text describes the importance and benefits of Mari

929 3610 15  ---SDG 15---
930 152 15  ---SDG 15---
931 658 15  ---SDG 15---

Explanation:

The text provided discusses the value of biodiversity and ecosystem services, and how they are assessed and integrated into development and poverty reduction strategies. The focus of the text is on the natural environment and its preservation, which aligns with SDG 15: Life on Land. SDG 15 aims to protect, restore and promote the sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss. Therefore, the label for this text is ---SDG 15---.
932 1672 15  ---SDG 15---
933 3170 15  ---SDG 15---
934 2084 15  ---SDG 15---
935 4203 15  ---SDG 15---
936 3947 15  ---SDG 15---

The text discusses the impact of agricultural expansion on forests, specifically in Cambodia where forest land has been cleared for ELCs (Economic Land Concessions) including mining, leading to a significant impact on forests. SDG 15 fo

965 4754 16  ---SDG 16---
966 930 16  ---SDG 16---
967 4430 16  ---SDG 16---
968 2761 16  ---SDG 16---
969 3839 16  ---SDG 16---
970 3501 16  ---SDG 16---
971 4998 16  ---SDG 16---
972 6738 16  ---SDG 16---
973 3540 16  ---SDG 16---
974 283 16  ---SDG 16---
975 4891 16  ---SDG 10---
976 3236 16  ---SDG 16---
977 6715 16  ---SDG 16---
978 2543 16  ---SDG 16---
979 64 16  ---SDG 16---
980 3614 16  ---SDG 16---
981 4520 16  ---SDG 3---
982 569 16  ---SDG 16---
983 543 16  ---SDG 16---
984 4331 16  ---SDG 16---
985 5742 16  ---SDG 16---
986 1384 16  ---SDG 16---

Explanation:

The text discusses the role of Christian missionaries and their successful efforts in planting churches around the world, particularly in sub-Saharan Africa. It then compares their strategy to that of contemporary international non-governmental organizations and inter-governmental organizations engaged in development, humanitarian assistance, peace-building, and human rights. The text suggests that an essential ingre

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9---')
('2', 6471, 0, ' ---SDG 16---')
('782', 2180, 13, ' ---SDG 13---')
('601', 4947, 10, " ---SDG 10---\n\n(Explanation: The text discusses the relationship between income inequality and economic growth, which aligns with SDG 10's focus on reducing inequality within and among countries. The example provided in the text, China, is a transition economy with fast economic growth, which further supports the classification under SDG 10.)")
('144', 2727, 2, ' ---SDG 2--- \\\\\nThis text discusses the need for a more radical and integrated policy approach to promote sustainable food production and reduce poverty, hunger, and environmental destruction. It highlights the importance of scaling up innovations, disseminating knowledge, building capacity among farmers, fostering trust, and reducing risks associated with new technology and agricultural practices. These themes align closely with SDG 2, which aims to end hunger, achieve f

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9---
2,2,6471,0,---SDG 16---
3,782,2180,13,---SDG 13---
4,601,4947,10,---SDG 10---\n\n(Explanation: The text discus...
...,...,...,...,...
1015,152,5700,2,---SDG 1---\n\nThe text emphasizes on the imp...
1016,276,1547,4,---SDG 4---
1017,554,5840,9,---SDG 9---\n\nThe text emphasizes the import...
1018,927,6479,15,---SDG 15---\n\n(SDG 15 is the Sustainable De...


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 0---
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 9---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 0---,0
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 9---,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,38,38,0,38,38,0,0,0,38,38,38,38,38
1,71,71,70,71,71,70,70,70,71,71,71,71,71
10,45,45,45,45,45,45,45,45,45,45,45,45,45
11,71,71,70,71,71,70,70,70,71,71,71,71,71
12,58,58,58,58,58,58,58,58,58,58,58,58,58
13,61,61,61,61,61,61,61,61,61,61,61,61,61
14,58,58,58,58,58,58,58,58,58,58,58,58,58
15,60,60,58,60,60,58,58,58,60,60,60,60,60
16,56,56,54,56,56,54,54,54,56,56,56,56,56


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.63      0.78        60
           1       0.73      0.87      0.79        60
           2       0.91      0.80      0.85        60
           3       0.84      0.93      0.88        60
           4       0.89      0.92      0.90        60
           5       0.88      0.93      0.90        60
           6       1.00      0.95      0.97        60
           7       0.93      0.92      0.92        60
           8       0.69      0.70      0.69        60
           9       0.72      0.90      0.80        60
          10       0.82      0.62      0.70        60
          11       0.82      0.97      0.89        60
          12       0.91      0.88      0.90        60
          13       0.90      0.92      0.91        60
          14       0.97      0.93      0.95        60
          15       0.93      0.93      0.93        60
          16       0.95      0.88      0.91        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.93      0.65      0.76        60
           1       0.73      0.87      0.79        60
           2       0.91      0.80      0.85        60
           3       0.84      0.93      0.88        60
           4       0.89      0.92      0.90        60
           5       0.88      0.93      0.90        60
           6       1.00      0.95      0.97        60
           7       0.93      0.92      0.92        60
           8       0.69      0.70      0.69        60
           9       0.72      0.90      0.80        60
          10       0.82      0.62      0.70        60
          11       0.82      0.97      0.89        60
          12       0.91      0.88      0.90        60
          13       0.90      0.92      0.91        60
          14       0.97      0.93      0.95        60
          15       0.93      0.93      0.93        60
          16       0.95      0.88      0.91        60

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,92.857143,65.000000,76.470588,60.000000
1,73.239437,86.666667,79.389313,60.000000
2,90.566038,80.000000,84.955752,60.000000
3,83.582090,93.333333,88.188976,60.000000
4,88.709677,91.666667,90.163934,60.000000
5,87.500000,93.333333,90.322581,60.000000
6,100.000000,95.000000,97.435897,60.000000
7,93.220339,91.666667,92.436975,60.000000
8,68.852459,70.000000,69.421488,60.000000
9,72.000000,90.000000,80.000000,60.000000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

87.08886497860114 86.47058823529412 86.47058823529412 86.36345945877429


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

87.1 86.5 86.5 86.4
